In [2]:
import mysql.connector
import pandas as pd

In [3]:
cnx = mysql.connector.connect(user='root', password='TO52',
                              host='localhost',
                              database='clothes_detection')

In [4]:
def construct_query(cat, brand, size):
    query = "SELECT absolute_path FROM clothes NATURAL JOIN category "
    where_category = f"WHERE category = '{cat}' "
    where_brand = ""
    where_size = ""
    first = True
    
    if brand != 'all':
        query = query + "NATURAL JOIN brand "
        for bd in brand:
            if first:
                where_brand = f"AND (brand = '{bd}' "
                first = False
            else:
                where_brand = where_brand + f"OR brand = '{bd}' "
        if len(brand) > 0:
            where_brand = where_brand + ") "

    first = True
    if size != 'all':
        query = query + "NATURAL JOIN size "
        for sz in size:
            if first:
                where_size = f"AND (size = '{sz}' "
                first = False
            else:
                where_size = where_size + f"OR size = '{sz}' "
        if len(size) > 0:
            where_size = where_size + ") "
    
    query = query + where_category + where_brand + where_size + "LIMIT 10"
    
    return query

In [5]:
def get_sizes_bdd():
    cursor = cnx.cursor()
    cursor.execute("SELECT DISTINCT size FROM size")
    sizes = cursor.fetchall()
    sizes = [size[0] for size in sizes]
    return sizes

In [6]:
sizes = get_sizes_bdd()
print(sizes)

['XXS', 'XS', 'S', 'M', 'L', 'XL', 'XXL']


In [7]:
def get_images_bdd(cat, brand, size):
    cursor = cnx.cursor()
    query = construct_query(cat, brand, size)
    print(query)
    cursor.execute(query)
    paths = cursor.fetchall()
    paths = [path[0] for path in paths]
    return paths

In [8]:
paths = get_images_bdd('Tee', ['Zara','H&M'], ['XS'])
print(paths)

SELECT absolute_path FROM clothes NATURAL JOIN category NATURAL JOIN brand NATURAL JOIN size WHERE category = 'Tee' AND (brand = 'Zara' OR brand = 'H&M' ) AND (size = 'XS' ) LIMIT 10
['img/Heathered_Muscle_Tee/img_00000008.jpg', 'img/Heathered_Mesh-Paneled_Colorblock_Tee/img_00000020.jpg', 'img/Heathered_Knit_Tee/img_00000025.jpg', 'img/Heathered_Frida_Kahlo_Tee/img_00000030.jpg', 'img/Heathered_Crew_Neck_Tee/img_00000061.jpg', 'img/Heathered_Crew_Neck_Tee/img_00000023.jpg', 'img/Heathered_Crew_Neck_Tee/img_00000019.jpg', 'img/Heathered_Colorblock_Raglan_Tee/img_00000027.jpg', 'img/Heathered_Colorblock_Raglan_Tee/img_00000012.jpg', 'img/Baseball_Muscle_Tee/img_00000023.jpg']


In [9]:
def get_category_bdd(cat):
    cursor = cnx.cursor()
    cursor.execute("SELECT category FROM category WHERE id_category = " + str(cat))
    category = cursor.fetchall()
    return category[0][0]

In [10]:
cat = get_category_bdd(2)
cat

'Blouse'

In [16]:
def get_size_and_brand(path):
    cursor = cnx.cursor()
    cursor.execute(f"SELECT size.size FROM clothes NATURAL JOIN size WHERE absolute_path = '{path}' ;")
    size= cursor.fetchall()
    cursor.execute(f"SELECT brand.brand FROM clothes NATURAL JOIN brand WHERE absolute_path = '{path}' ;")
    brand= cursor.fetchall()
    return size[0][0], brand[0][0]

In [17]:
path = 'img/Crochet_Cami_Strap_Dress/img_00000020.jpg'
size, brand = get_size_and_brand(path)
print(size, brand)

M Asos


In [ ]:
cursor = cnx.cursor()
brand = ['Zara','H&M']
query = construct_query('Tee', brand ,['XS'])
print(query)

cursor.execute(query)

df = pd.DataFrame(cursor.fetchall())
df.columns = cursor.column_names

print(df)

In [ ]:
cursor = cnx.cursor()

cat = 'Dress'
brand = ['Zara']
size = ['M','L']

query = construct_query(cat, brand, size)
print(query)

cursor.execute(query)

df = pd.DataFrame(cursor.fetchall())
df.columns = cursor.column_names

print(df)


In [ ]:
cnx.close()